# Import

In [ ]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from source.Prototype1 import Prototype1

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import multiprocessing
import torch
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import denoise_wavelet
import random
import math
from PIL import Image
from collections import deque
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, label, generate_binary_structure
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.ndimage import label as ndi_label, binary_dilation

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Init Variables

In [ ]:
model = Prototype1(num_attention_heads=16).to("cuda")
if torch.cuda.is_available():
    print("Using cuda cores")
    model.cuda()

dataset = NEUDataset(set="train", seed=555, scale=0.5)

# Hyper parameters

In [4]:
LEARNING_RATE = 0.0005
EPOCH = 1000
BATCH_SIZE = len(dataset)//2
THREADS = 8

# Training process

In [5]:
criterion = nn.MSELoss(reduction="sum")
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS)

In [ ]:
def train(model):
    loss_record = []
    for epoch in range(EPOCH):
        model.train()
        total_loss = 0
        num_batches = 0

        # Desempaquetar ignorando 'labels'
        for images, _, best_parameters in dataloader:
            images = images.to('cuda')
            best_parameters = best_parameters.to('cuda')

            # Forward: el modelo predice sp y sr
            pred_reg = model(images)  # salida de regresión

            # Calcular la pérdida de regresión
            loss = criterion(pred_reg, best_parameters)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            num_batches += 1

        avg_loss = total_loss / num_batches
        loss_record.append(avg_loss)
        print(f"Epoch [{epoch + 1}/{EPOCH}], Loss: {avg_loss:.4f}")
    return loss, model

loss, model = train(model)

In [ ]:
plt.plot(loss)           # Dibuja gráfico de línea
plt.title("Line Chart")  # Título del gráfico
plt.xlabel("Índice")     # Etiqueta eje x
plt.ylabel("Valor")      # Etiqueta eje y
plt.show()               # Muestra el gráfico

In [7]:
# Guardar solo la red neuronal en un archivo .pth
#torch.save(model.state_dict(), "h1.pth")
#print("Modelo guardado exitosamente en h1.pth")

In [ ]:
for i in range(len(dataset)):
    image , label, best_parameters = dataset.__getitem__(index=i)
    values  = model(image.to("cuda"))

    sp = float(values[0][0].detach().cpu().numpy())
    sr = float(values[0][1].detach().cpu().numpy())

    print("sp", sp)
    print("sr", sr)